In [1]:
import pandas as pd
import numpy as np
import processing.scales as scales
import os

# define variable names
tia_scales = scales.tia_scales
hcsds_scales = scales.hcsds_scales
ati_scales = scales.ati_scales
manip_check_scales = scales.manip_check_scales

scale_titles = scales.scale_titles

plots_path = '../plots/univariate_analysis/'

# Create output directory if it doesn't exist
output_path = '../output/'
os.makedirs(output_path, exist_ok=True)

# Load data
data = pd.read_csv('../data/data_scales.csv')
print(f"Total sample size: {len(data)}")
print(f"\nGroup distribution:")
print(data['stimulus_group'].value_counts())
print('\n')

# compute basic statistics for later tests
N = len(data)
p = len(tia_scales)
k = data['stimulus_group'].nunique()

# set alpha and power
alpha = 0.05
power = 0.80

print(f'Sample size: N = {N}')
print(f'Number of outcomes: p = {p}')
print(f'Number of groups: k = {k}')

Total sample size: 255

Group distribution:
stimulus_group
1    129
0    126
Name: count, dtype: int64


Sample size: N = 255
Number of outcomes: p = 5
Number of groups: k = 2


In [2]:
# helper function for APA-style p-value reporting
def apa_p(p, sig_stars=False):
    """Format p-value in APA style (no leading zero)."""
    def apa_sig(p):
        """Return significance stars."""
        if p < 0.001:
            return "***"
        elif p < 0.01:
            return "**"
        elif p < 0.05:
            return "*"
        else:
            return ""

    output = ''
    if p < 0.001:
        output =  "< .001"
    else:
        output =  f"{p:.3f}".replace("0.", ".")

    if sig_stars:
        output += ' ' + apa_sig(p)

    return output


## Data Preparation

### Variable Centering and Coding

We prepare variables for moderation analysis:

1. **Effect code treatment**: stimulus_group as -0.5 (control) and 0.5 (uncertainty)
2. **Standardize continuous variables**: For better comparison of beta values between variables
3. **Effect code categorical variables**: For symmetric interpretation

In [3]:
# 1. Effect code treatment: control = -0.5, uncertainty = 0.5
data['group_effect'] = data['stimulus_group'] - 0.5

# 2. Normalize all continuous variables
continuous_vars = hcsds_scales + ati_scales + ['age', 'page_submit']

for var in continuous_vars:
    data[f'{var}_c'] = (data[var] - data[var].mean())/data[var].std()

# 3. Effect code gender: male (1) = 0.5, female (2) = -0.5, "other/prefer not to say" (3) = 0
data['gender_c'] = data['gender'].map({1: 0.5, 2: -0.5, 3: 0})

# 4. Mean-center ordinal variables (education, AI experience)
data['education_c'] = data['education'] - data['education'].mean()
data['ai_exp_c'] = data['Q19'] - data['Q19'].mean()
data.drop('Q19', axis=1)

demographics = ['age', 'gender', 'education', 'ai_exp']

demographics_c = [f'{s}_c' for s in demographics]
hcsds_scales_c = [f'{s}_c' for s in hcsds_scales]
ati_scales_c = [f'{s}_c' for s in ati_scales]

print(f"Prepared {len(data)} observations for analysis")
print(f"Continuous moderators: {len(continuous_vars)}")
print(f"Total moderators to test: {len(continuous_vars) + 3}")  # + gender, education, Q19

Prepared 255 observations for analysis
Continuous moderators: 5
Total moderators to test: 8


## ANOVA Models Selection

In [4]:
# make strings for list of predictors in each model
model_predictors = []
model_predictors.append(['group_effect'])
model_predictors.append(model_predictors[0] + demographics_c)
model_predictors.append(model_predictors[1] + hcsds_scales_c + ati_scales_c)

interactions = [f'group_effect:{p}' for p in demographics_c + hcsds_scales_c + ati_scales_c]

model_predictors.append(model_predictors[2] + interactions)

# build models for each tia_subscale separately
import statsmodels.formula.api as smf

results = {}
for subscale in tia_scales:
    results[subscale] = {}
    for model_nr, predictors in enumerate(model_predictors):
        formula = f'{subscale} ~ {' + '.join(predictors)}'
        result = smf.ols(formula, data=data).fit()

        print(f'Predicting {subscale} in model {model_nr+1}')
        print(result.summary())
        print(f'{'='*20}\n\n')

        results[subscale][model_nr+1] = result



Predicting tia_f in model 1
                            OLS Regression Results                            
Dep. Variable:                  tia_f   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.675
Date:                Fri, 28 Nov 2025   Prob (F-statistic):              0.197
Time:                        17:06:26   Log-Likelihood:                -338.80
No. Observations:                 255   AIC:                             681.6
Df Residuals:                     253   BIC:                             688.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        2.1

### Conclusion Model Selection
Select full model, unless I get other information from SMAP.

In [7]:
SELECT_MODEL = 4
CORRECTION_METHOD = "holm"
################################

result_tables = {}

# save results to tables
for subscale in tia_scales:
    df = pd.DataFrame(results[subscale][SELECT_MODEL].summary().tables[1])
    df.iloc[0, 0] = 'predictor'
    df.columns = ['predictor', 'coef', 'std err', 't', 'p', 'CI_lower', 'CI_higher']
    df = df[1:]

    result_tables[subscale] = df

    # save to file
    df.to_csv(f'{output_path}{subscale}_regression_coef.csv', index=False)

'''
Somehow, the `tables` object from `statsmodels` gives values as a `Cell` object instead of just numbers (wtf?!).
It's therefore easiest to just save everything to a file and then reading it again (very stupid, I know).
'''

# correct p-values for multiple comparisons
from statsmodels.stats.multitest import multipletests

for subscale in tia_scales:
    result_table = pd.read_csv(f'{output_path}{subscale}_regression_coef.csv')

    if CORRECTION_METHOD.lower() == 'none':
        result_table['p_adj'] = result_table['p'].copy()
    else:
        result_table['p_adj'] = multipletests(result_table['p'],
                                        alpha=alpha,
                                        method=CORRECTION_METHOD)[1]

    sig = []
    p_adj_report = []
    for i, row in result_table.iterrows():
        p_adj_report.append(apa_p(row['p_adj'],
                                    sig_stars=True))
    result_table['p_adj_report'] = p_adj_report

    # save to file, overwrite original csv file
    result_table.to_csv(f'{output_path}{subscale}_regression_coef.csv', index=False)


## 5b. Univariate Non-Inferiority Test (ANOVA)

In [ ]:
# find MDE (minimally detectable effect) in a ANOVA
from scipy import stats

def mde_coefficient(se, df, alpha=0.05, power=0.80):
    """
    Minimally detectable effect for a regression coefficient.

    Parameters:
        se: standard error of the coefficient (from statsmodels)
        df: residual degrees of freedom (model.df_resid)
        alpha: significance level (two-tailed)
        power: desired power

    Returns:
        MDE in original units of the coefficient
    """
    t_crit = stats.t.ppf(1 - alpha/2, df)  # two-tailed significance
    t_pow = stats.t.ppf(power, df)         # power threshold
    return (t_crit + t_pow) * se

In [ ]:
# compute non-inferiority test in ANOVA for each outcome variable